In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# from bokeh.io import output_file, show
# from bokeh.models import ColumnDataSource, GMapOptions
# from bokeh.plotting import gmap
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

import plotly
import plotly.offline as po
plotly.offline.init_notebook_mode(connected=True)
import plotly.plotly as py
import plotly.graph_objs as go

import ipywidgets as widgets
from ipywidgets import interactive, HBox, VBox
%matplotlib notebook

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
business_data = pd.read_csv('restaurant_data1', sep=',')

In [4]:
business_data.head(2)

,Name,Star_Rating,Number_Of_Reviewers,Price,Price_Range,Category,Phone,Website,Open_Hours,MonBH,...,Liked by Vegetarians,Offers Military Discount,Open to All,Outdoor Seating,Smoking,Take-out,Takes Reservations,Waiter Service,Wheelchair Accessible,Wi-Fi
0,Ambar,4.5,824 reviews,$$,$11-30,ModernEuropean,(703) 875-9663,ambarrestaurant.com/home-page…,Hours\r\n \r\n \r\n ...,14.0,...,1,0,0,1,0,1,1,1,1,0
1,Mejana,4.0,582 reviews,$$,$11-30,"Mediterranean,Lebanese",(703) 465-4440,mejanarestaurant.com,Hours\r\n \r\n \r\n ...,10.5,...,1,1,0,1,0,1,1,0,0,1


In [5]:
business_data.columns
business_data.drop(columns=['Price', 'Category', 'Phone', 'Website', 'Open_Hours'], axis=1, inplace=True)

In [6]:
business_data.loc[ business_data.Number_Of_Reviewers == '1 review', 'Number_Of_Reviewers'] = '1 reviews'
business_data['Number_Of_Reviewers'].replace(regex=True,inplace=True,to_replace=r' reviews',value=r'')
business_data.rename(columns={'Number_Of_Reviewers':'Number_of_Reviewers'}, inplace=True)
cols = ['Number_of_Reviewers']
business_data[cols] = business_data[cols].apply(pd.to_numeric)

In [7]:
encoder = LabelEncoder()
business_data['PriceRange'] = business_data.Price_Range
business_data.Price_Range = encoder.fit_transform(business_data.Price_Range)

In [8]:
mapbox_access_token = 'pk.eyJ1IjoiZHJlYW15anBsIiwiYSI6ImNqbzFoNm00YjA2Z2kza3FuOWFmN2I3NXoifQ.cADYKcSYogjVVHBiNYvY9A'

In [47]:
def plt_gmap(center, lat, lon, name, size, color):
    data = [
        go.Scattermapbox(
            lat=lat,
            lon=lon,
            mode='markers',
            marker=dict(
                size=size,
                color=color,
                colorscale='Jet',
                colorbar = dict(
                    title = '{}'.format(color.name)),
                opacity = 0.6
            ),
            text=name,
        )
    ]

    layout = go.Layout(
        autosize=True,
        hovermode='closest',
#         height=512,
#         width=400,
        mapbox=dict(
            accesstoken=mapbox_access_token,
            bearing=0,
            center=dict(
                lat=center[0],
                lon=center[1]
            ),
            pitch=0,
            zoom=6,
        ),
        margin=dict(
            l=40,
            r=40,
            b=40,
            t=40
        )
    )

    fig = dict(data=data, layout=layout)
    return fig

In [48]:
center = [36.84, -76.1]
center = [37.9407, -77.4360]

lat=list(business_data['Latitude'].values)
lon=list(business_data['Longitude'].values)
name = business_data['Name']
color = business_data['Star_Rating']
# size = (size-size.min())/(size.max()-size.min())
size = (color / color.max()) * 50

In [49]:
light_mode = 'mapbox://styles/mapbox/light-v9'
basic_mode = 'mapbox://styles/mapbox/streets-v10'
satellite_streets_mode = 'mapbox://styles/mapbox/satellite-streets-v9'
fig_type = [light_mode, basic_mode, satellite_streets_mode]

In [50]:
business_data

,Name,Star_Rating,Number_of_Reviewers,Price_Range,MonBH,TueBH,WedBH,ThuBH,FriBH,SatBH,...,Offers Military Discount,Open to All,Outdoor Seating,Smoking,Take-out,Takes Reservations,Waiter Service,Wheelchair Accessible,Wi-Fi,PriceRange
0,Ambar,4.5,824,0,14.00,14.00,14.00,14.00,15.00,13.50,...,0,0,1,0,1,1,1,1,0,$11-30
1,Mejana,4.0,582,0,10.50,10.50,10.50,10.50,11.50,11.50,...,1,0,1,0,1,1,0,0,1,$11-30
2,Barley Mac,4.0,485,0,15.00,15.00,15.00,15.00,15.00,15.50,...,0,0,1,0,1,1,1,0,1,$11-30
3,Green Pig Bistro,4.0,781,0,10.50,10.50,11.00,1.00,14.00,14.50,...,0,0,1,0,0,1,0,0,1,$11-30
4,Tupelo Honey,4.0,1312,0,11.00,11.00,11.00,11.00,12.00,14.00,...,0,0,1,0,1,1,1,1,1,$11-30
5,Urban Tandoor,4.0,133,0,10.25,10.25,10.25,10.25,11.25,11.25,...,0,0,1,0,1,1,1,0,1,$11-30
6,Baba,4.5,211,0,20.00,20.00,20.00,20.00,10.00,16.50,...,0,0,0,0,1,1,1,1,1,$11-30
7,Yayla Bistro,4.5,401,0,11.00,11.00,11.00,11.00,11.00,11.00,...,0,0,1,0,1,1,1,1,1,$11-30
8,Four Sisters Grill,4.0,467,3,10.00,10.00,10.00,10.00,10.00,10.00,...,0,0,1,0,1,0,0,0,0,Under $10
9,The Liberty Tavern,4.0,1170,0,14.50,14.50,14.50,14.50,14.50,16.50,...,0,0,1,0,1,1,1,1,1,$11-30


In [51]:
fig = plt_gmap(center, lat, lon, name, size, color)
f = go.FigureWidget(fig)

def update_color_size(color, size, fig_type):
    scatter = f.data[0]
    layout = f.layout
#     business_data[xaxis] = (business_data[xaxis]-business_data[xaxis].min())/(
#         business_data[xaxis].max()-business_data[xaxis].min())
#     business_data[yaxis] = (business_data[yaxis]-business_data[yaxis].min())/(
#         business_data[yaxis].max()-business_data[yaxis].min())

    scatter.marker.color = business_data[color]
#     scatter.marker.size = (business_data[size]-business_data[size].min())/(
#         business_data[size].max()-business_data[size].min()) * 50
    scatter.marker.size = (business_data[size]/business_data[size].max())*40
    scatter.marker.colorbar.title = '{}'.format(color)
    layout.mapbox.style = '{}'.format(fig_type)

axis_dropdowns = interactive(update_color_size, 
                             size = business_data.select_dtypes('number').columns, 
                             color = business_data.select_dtypes('number').columns,
                            fig_type = fig_type)

t = go.FigureWidget([go.Table(
    header=dict(values=['Name','Star_Rating','Number_of_Reviewers', 'PriceRange'],
                fill = dict(color='#C2D4FF'),
                align = ['left'] * 5),
    cells=dict(values=[business_data[col] for col in ['Name','Star_Rating','Number_of_Reviewers', 'PriceRange']],
               fill = dict(color='#F5F8FF'),
               align = ['left'] * 5))]
                   )


def selection_fn(trace,points,selector):
    t.data[0].cells.values = [business_data.loc[points.point_inds][col]
                              for col in ['Name','Star_Rating','Number_of_Reviewers', 'PriceRange']]

f.data[0].on_selection(selection_fn)

# Put everything together
u = VBox((HBox(axis_dropdowns.children),f,t))

In [52]:
t.layout.height = 575

In [53]:
display(u)

In [22]:
business_data_plt = business_data[business_data.select_dtypes('number').columns[:-2]]
rndperm = np.random.permutation(business_data_plt.shape[0])
X_business_data_plt = business_data_plt.drop(business_data_plt.columns[0], axis=1)
y_business_data_plt = business_data_plt[business_data_plt.columns[0]]

pca = PCA(n_components=3)
pca_result = pca.fit_transform(X_business_data_plt.values)
pca.explained_variance_ratio_

array([9.93971946e-01, 3.54340243e-03, 9.38501702e-04])

In [23]:
X_business_data_plt_embedded = TSNE(n_components=3, 
                                    perplexity=20,
                                    learning_rate=0.5,
                                    n_iter=20000,
                                    random_state=42).fit_transform(X_business_data_plt)

In [32]:
business_data_tsne_embedded = go.Scatter3d(
    x=X_business_data_plt_embedded[:,0],
    y=X_business_data_plt_embedded[:,1],
    z=X_business_data_plt_embedded[:,2],
    mode='markers',
    marker=dict(
        size=12,
        color=y_business_data_plt,  # set color to an array/list of desired values
        colorscale='Jet',       # choose a colorscale
        opacity=0.8,
        colorbar = dict(title = '{}'.format(y_business_data_plt.name))
    ),
    text = y_business_data_plt
)

data = [business_data_tsne_embedded]
layout = go.Layout(
     scene = dict(xaxis=dict(title='tsne[:,0]'),
                            yaxis=dict(title='tsne[:,1]'),
                            zaxis=dict(title='tsne[:,2]'),
                           ),
#     height=1080,
    height=600,
#     width = 1750,
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = go.Figure(data=data, layout=layout)
po.iplot(fig, filename='3d-scatter-colorscale')